# Purpose
We are in the process of converting some functions in `pudl.output` to be SQL views. This notebook allows us to compare the outputs of the old python functions with the SQL view.

In [1]:
import os

assert os.environ.get("DAGSTER_HOME"), (
    "The DAGSTER_HOME env var is not set so dagster won't be able to find the assets."
    "Set the DAGSTER_HOME env var in this notebook or kill the jupyter server and set"
    " the DAGSTER_HOME env var in your terminal and relaunch jupyter."
)

In [2]:
from pudl.workspace.setup import get_defaults
import sqlalchemy as sa
import pandas as pd

# TODO: This should be replaced with get_defaults()
engine = sa.create_engine(f"sqlite:///{os.environ['PUDL_OUTPUT']}pudl.sqlite")

pudl_settings is being deprecated in favor of environment variables variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet directories are no longer being used. Make sure there is a single directory named 'output' at the root of your workspace. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet direct

## Compare output of old python function with SQL view

In [3]:
%%time
view_name = "denorm_plants_utils_ferc1"

with engine.connect() as con:
    view_df = pd.read_sql_table(view_name, con)

CPU times: user 34.1 ms, sys: 3.71 ms, total: 37.8 ms
Wall time: 53.8 ms


In [4]:
view_df.head()

,utility_id_ferc1,plant_name_ferc1,plant_id_pudl,utility_name_ferc1,utility_id_pudl
0,206,*dolet hills (3),1,Southwestern Electric Power Company,301
1,206,*flint creek (1),2,Southwestern Electric Power Company,301
2,206,*pirkey (2),3,Southwestern Electric Power Company,301
3,159,59th st gt-1,4,"Consolidated Edison Company of New York, Inc.",79
4,159,74th st gt 1&2,5,"Consolidated Edison Company of New York, Inc.",79


In [5]:
# Import the old python functions
from pudl.output.ferc1 import plants_utils_ferc1

old_output_func = plants_utils_ferc1

old_df = old_output_func(engine)

# Align pandas index and sort values using all of the columns
key = list(old_df.columns)
old_df = old_df.sort_values(by=key).reset_index(drop=True)
view_df = view_df.sort_values(by=key).reset_index(drop=True)

pd.testing.assert_frame_equal(old_df, view_df)

2023-03-23 13:35:06 [ WARNING] catalystcoop.pudl.output.ferc1:56 pudl.output.ferc1.plants_utils_ferc1() will be deprecated in a future version of PUDL. In the future, call the PudlTabl.pu_ferc1() method or pull the plants_utils_ferc1 tabledirectly from the pudl.sqlite database.


## Make sure we can load the view using the SQLite IO Manager

In [6]:
from dagster import AssetKey

from pudl.etl import defs

df = defs.load_asset_value(AssetKey(view_name))

df.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   utility_id_ferc1    5 non-null      Int64 
 1   plant_name_ferc1    5 non-null      string
 2   plant_id_pudl       5 non-null      Int64 
 3   utility_name_ferc1  5 non-null      string
 4   utility_id_pudl     5 non-null      Int64 
dtypes: Int64(3), string(2)
memory usage: 343.0 bytes
